In [40]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

In [41]:
# 加载训练集
train = pd.read_csv('train_dataset_v2.tsv', sep='\t', error_bad_lines=False, warn_bad_lines=False)
print(train.shape)
train.head()

(42790, 4)


,id,content,character,emotions
0,1171_0001_A_1,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,o2,"0,0,0,0,0,0"
1,1171_0001_A_2,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。,c1,"0,0,0,0,0,0"
2,1171_0001_A_3,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,o2,"0,0,0,0,0,0"
3,1171_0001_A_4,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。,c1,"0,0,0,0,0,0"
4,1171_0001_A_5,o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...,o2,"0,0,0,0,0,0"


In [42]:
# 加载测试集
test = pd.read_csv('test_dataset.tsv', sep='\t')
print(test.shape)
test.head()

(21376, 3)


,id,content,character
0,34170_0002_A_12,穿着背心的b1醒来，看看手机，三点了。,b1
1,34170_0002_A_14,b1走出卧室。,b1
2,34170_0003_A_16,b1拿着手机，点开计时功能。,b1
3,34170_0003_A_17,b1站在淋浴头下面，水从b1的头和脸上冲刷而过。,b1
4,34170_0003_A_18,b1摈着呼吸。,b1


In [43]:
submit = pd.read_csv('submit_example.tsv', sep='\t')
print(submit.shape)
submit.head()

(21376, 2)


,id,emotion
0,34170_0002_A_12,"0,0,0,0,0,0"
1,34170_0002_A_14,"0,0,0,0,0,0"
2,34170_0003_A_16,"0,0,0,0,0,0"
3,34170_0003_A_17,"0,0,0,0,0,0"
4,34170_0003_A_18,"0,0,0,0,0,0"


In [44]:
train = train[train['emotions'].notna()]
print(train.shape)

(36782, 4)


In [45]:
def get_context(df, cur_id):
    pos = cur_id.split('_')[-1]
    # print(pos)
    pos_str = str(int(pos) - 1)

    location = train['id'][0][:-2] + '_' + pos_str
    #print(location)
    ret = train.loc[train['id'] == location]['content'].values.tolist()
    if len(ret) == 1:
        return ret[0]
    else:
        return "无上文"
print(get_context(train, '1171_0001_A_1'))

无上文


In [46]:
train['character'].fillna('无角色', inplace=True)
test['character'].fillna('无角色', inplace=True)

train['text'] = train['content'].astype(str) + ' 角色: ' + train['character'].astype(str)
test['text'] = test['content'].astype(str) + ' 角色: ' + test['character'].astype(str)

In [47]:
# add train context info
for i in range(len(train['text'])):
    try:
        lcw = get_context(train, train['id'][i])
    except:
        continue
    train['text'][i] += ' 上文: ' + lcw
train.head

<bound method NDFrame.head of                       id                                            content  \
0          1171_0001_A_1          天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。   
1          1171_0001_A_2          天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。   
2          1171_0001_A_3                       o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。   
3          1171_0001_A_4                       o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。   
4          1171_0001_A_5  o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...   
...                  ...                                                ...   
42785  34949_0180_A_1450                                x2的未婚夫手捧大束玫瑰花在寻找x2。   
42786  34949_0180_A_1458                f1看看表：过了五分钟。他从口袋里掏出药盒，倒出药：一片，两片，半片。   
42787  34949_0180_A_1459  b1转脸朝前走。突然站住了，他看见自己的前妻，c1领着女儿g3站在前面。他过去，三人紧紧拥在...   
42788  34949_0180_A_1460  b1转脸朝前走。突然站住了，他看见自己的前妻，c1领着女儿g3站在前面。他过去，三人紧紧拥在...   
42789  34949_0180_A_1463  b1转脸朝前走。突然站住了，他看见自己的前妻，c1领着女儿g3站在前面。他过去，三人紧紧拥在...   

      character     e

In [48]:
# add test context info
for i in range(len(test['id'])):
    try:
        lcw = get_context(test, test['id'][i])
    except:
        continue
    test['text'][i] += ' 上文: ' + lcw
test.head

<bound method NDFrame.head of                       id                   content character  \
0        34170_0002_A_12       穿着背心的b1醒来，看看手机，三点了。        b1   
1        34170_0002_A_14                   b1走出卧室。        b1   
2        34170_0003_A_16            b1拿着手机，点开计时功能。        b1   
3        34170_0003_A_17  b1站在淋浴头下面，水从b1的头和脸上冲刷而过。        b1   
4        34170_0003_A_18                   b1摈着呼吸。        b1   
...                  ...                       ...       ...   
21371  34122_0130_A_1184               w2：正版药进医保了。        w2   
21372  34122_0130_A_1185                   u2：那挺好。        u2   
21373  34122_0130_A_1186      w2：你还是接着卖你的壮阳药吧，适合你。        w2   
21374  34122_0130_A_1187           w2开着车，带着u2驶向远方。        u2   
21375  34122_0130_A_1188           w2开着车，带着u2驶向远方。        w2   

                                                    text  
0      穿着背心的b1醒来，看看手机，三点了。 角色: b1 上文: o2笑了笑：军礼不是这么敬的。...  
1                     b1走出卧室。 角色: b1 上文: o2示范了一个动作，c1照做。  
2                       

In [49]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [50]:
# 失败的尝试，直接拼 0.69 -> 0.68
# train['text'] = train['context'].astype(str) + train['text'].astype(str)
# test['text'] = test['context'].astype(str) + test['text'].astype(str)
print(train.shape)

(36782, 5)


In [51]:
train['labels'] = train['emotions'].apply(lambda x: [int(i) for i in x.split(',')])

In [52]:
train_data = train[['text', 'labels']].copy()
train_data.head()

,text,labels
0,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: ...,"[0, 0, 0, 0, 0, 0]"
1,天空下着暴雨，o2正在给c1穿雨衣，他自己却只穿着单薄的军装，完全暴露在大雨之中。 角色: ...,"[0, 0, 0, 0, 0, 0]"
2,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: o2 上文: 天空下着暴雨...,"[0, 0, 0, 0, 0, 0]"
3,o2一手拿着一个行李，一路小跑着把c1带到了文工团门口。 角色: c1 上文: o2一手拿着...,"[0, 0, 0, 0, 0, 0]"
4,o2停下来接过c1手里的行李：你妈妈交待我了，等领了军装一定要照张相寄回去，让街坊邻居都知道...,"[0, 0, 0, 0, 0, 0]"


In [53]:
train_data = train_data.sample(frac=1.0, random_state=42)
dev_data = train_data.iloc[30000:]

In [54]:
print(len(train_data))

36782


In [55]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [56]:
model_args = MultiLabelClassificationArgs()
model_args.max_seq_length = 128
model_args.num_train_epochs = 5
model_args.learning_rate = 2e-5
model_args.do_lower_case = True
model_args.overwrite_output_dir = True
model_args.no_save = True
model_args.save_model_every_epoch = False
model_args.save_steps = -1

In [57]:
print(model_args)

MultiLabelClassificationArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=True, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=True, gradient_accumulation_steps=1, learning_rate=2e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=128, m

In [58]:
#model = MultiLabelClassificationModel('bert', 'hfl/chinese-bert-wwm-ext', num_labels=6)
model = MultiLabelClassificationModel('bert', 'hfl/chinese-roberta-wwm-ext', num_labels=6)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification

In [59]:
model.train_model(train_data)

  0%|          | 0/36782 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4598 [00:00<?, ?it/s]

(4598, 0.2584208245440497)

In [60]:
# eval
predictions, raw_outputs = model.predict([text for text in dev_data['text'].values])
raw_outputs[raw_outputs<=0.01] = 0

from sklearn.metrics import mean_squared_error
result = raw_outputs.tolist()
# for i in range(len(result)):
#     for j in range(6):
#         if result[i][j] > 0.99:
#             result[i][j] += 0.5
test_y = dev_data['labels'].values.tolist()
rmse = np.sqrt(mean_squared_error(result, test_y))
score = 1.0 / (1+rmse)
print(f"得分：{score}")


  0%|          | 0/6782 [00:00<?, ?it/s]

  0%|          | 0/848 [00:00<?, ?it/s]

得分：0.7046656332279972


In [52]:
# # submit tag
# sub = submit.copy()
# sub['emotion'] = predictions
# sub['emotion'] = sub['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
# sub.head()
# sub.to_csv('4epoch_2e05_tags_roberta.tsv', sep='\t', index=False)

In [61]:
# test
predictions, raw_outputs = model.predict([text for text in test['text'].values])
raw_outputs[raw_outputs<=0.01] = 0
# sub probs
prob_submit = submit.copy()
to_submit = []
# prob_submit['emotion'] = raw_outputs
for i in range(len(raw_outputs)):
    lst = raw_outputs[i].tolist()
#     for j in range(6):
#         if lst[j] > 0.99:
#             lst[j] += 0.5
    to_submit.append(','.join([str(i) for i in lst]))
    # break
prob_submit['emotion'] = to_submit #prob_submit['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
prob_submit.head()
prob_submit.to_csv('0924_s7046_roberta_structured.tsv', sep='\t', index=False)

  0%|          | 0/21376 [00:00<?, ?it/s]

  0%|          | 0/2672 [00:00<?, ?it/s]

In [54]:
# # sub probs
# prob_submit = sub.copy()
# to_submit = []
# # prob_submit['emotion'] = raw_outputs
# for i in range(len(raw_outputs)):
#     lst = raw_outputs[i].tolist()
#     to_submit.append(','.join([str(i * 3) for i in lst]))
#     # break
# prob_submit['emotion'] = to_submit #prob_submit['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
# prob_submit.head()
# prob_submit.to_csv('4epoch_2e05_probs_emplified_roberta.tsv', sep='\t', index=False)

In [62]:
model = MultiLabelClassificationModel('bert', 'outputs', num_labels=6)

In [63]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
train_data_0 = []
train_data_1 = []
train_data_2 = []
train_data_3 = []
train_data_4 = []
train_data_5 = []

for i in range(len(train_data['text'])):
    try:
        ## train data
        train_data_0.append([train_data['text'][i], train_data['labels'][i][0]])
        train_data_1.append([train_data['text'][i], train_data['labels'][i][1]])
        train_data_2.append([train_data['text'][i], train_data['labels'][i][2]])
        train_data_3.append([train_data['text'][i], train_data['labels'][i][3]])
        train_data_4.append([train_data['text'][i], train_data['labels'][i][4]])
        train_data_5.append([train_data['text'][i], train_data['labels'][i][5]])
    except:
        continue

dev_data_0 = []
dev_data_1 = []
dev_data_2 = []
dev_data_3 = []
dev_data_4 = []
dev_data_5 = []

for i in range(len(dev_data['text'])):
    try:
        ## train data
        dev_data_0.append([dev_data['text'][i], dev_data['labels'][i][0]])
        dev_data_1.append([dev_data['text'][i], dev_data['labels'][i][1]])
        dev_data_2.append([dev_data['text'][i], dev_data['labels'][i][2]])
        dev_data_3.append([dev_data['text'][i], dev_data['labels'][i][3]])
        dev_data_4.append([dev_data['text'][i], dev_data['labels'][i][4]])
        dev_data_5.append([dev_data['text'][i], dev_data['labels'][i][5]])
    except:
        continue

In [64]:
train_df_0 = pd.DataFrame(train_data_0)
train_df_0.columns = ["text", "labels"]
train_df_1 = pd.DataFrame(train_data_1)
train_df_1.columns = ["text", "labels"]
train_df_2 = pd.DataFrame(train_data_2)
train_df_2.columns = ["text", "labels"]
train_df_3 = pd.DataFrame(train_data_3)
train_df_3.columns = ["text", "labels"]
train_df_4 = pd.DataFrame(train_data_4)
train_df_4.columns = ["text", "labels"]
train_df_5 = pd.DataFrame(train_data_5)
train_df_5.columns = ["text", "labels"]

In [65]:
dev_df_0 = pd.DataFrame(dev_data_0)
dev_df_0.columns = ["text", "labels"]
dev_df_1 = pd.DataFrame(dev_data_1)
dev_df_1.columns = ["text", "labels"]
dev_df_2 = pd.DataFrame(dev_data_2)
dev_df_2.columns = ["text", "labels"]
dev_df_3 = pd.DataFrame(dev_data_3)
dev_df_3.columns = ["text", "labels"]
dev_df_4 = pd.DataFrame(dev_data_4)
dev_df_4.columns = ["text", "labels"]
dev_df_5 = pd.DataFrame(dev_data_5)
dev_df_5.columns = ["text", "labels"]


In [66]:
# Optional model configuration
model_args_0 = ClassificationArgs(num_train_epochs=4, output_dir='outputs_0')

# Create a ClassificationModel
model_0 = ClassificationModel(
    'bert',
    'hfl/chinese-roberta-wwm-ext',
    num_labels=4,
    args=model_args_0
) 
# Train the model
model_0.train_model(train_df_0)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

  0%|          | 0/31787 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_4_2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Optional model configuration
model_args_1 = ClassificationArgs(num_train_epochs=4, output_dir='outputs_1')

# Create a ClassificationModel
model_1 = ClassificationModel(
    'bert',
    'hfl/chinese-roberta-wwm-ext',
    num_labels=4,
    args=model_args_1
) 
# Train the model
model_1.train_model(train_df_1)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

  0%|          | 0/31787 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_4_2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Running Epoch 2 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [68]:
# Optional model configuration
model_args_2 = ClassificationArgs(num_train_epochs=4, output_dir='outputs_2')

# Create a ClassificationModel
model_2 = ClassificationModel(
    'bert',
    'hfl/chinese-roberta-wwm-ext',
    num_labels=4,
    args=model_args_2
) 
# Train the model
model_2.train_model(train_df_2)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

(15896, 0.22567643049037953)

In [69]:
# Optional model configuration
model_args_3 = ClassificationArgs(num_train_epochs=4, output_dir='outputs_3')

# Create a ClassificationModel
model_3 = ClassificationModel(
    'bert',
    'hfl/chinese-roberta-wwm-ext',
    num_labels=4,
    args=model_args_3
) 
# Train the model
model_3.train_model(train_df_3)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

  0%|          | 0/31787 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_4_2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs_3.


(15896, 0.32029314788462837)

In [ ]:
# Optional model configuration
model_args_4 = ClassificationArgs(num_train_epochs=4, output_dir='outputs_4')

# Create a ClassificationModel
model_4 = ClassificationModel(
    'bert',
    'hfl/chinese-roberta-wwm-ext',
    num_labels=4,
    args=model_args_4
) 
# Train the model
model_4.train_model(train_df_4)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

  0%|          | 0/31787 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_4_2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/3974 [00:00<?, ?it/s]

In [ ]:
# Optional model configuration
model_args_5 = ClassificationArgs(num_train_epochs=4, output_dir='outputs_5')

# Create a ClassificationModel
model_5 = ClassificationModel(
    'bert',
    'hfl/chinese-roberta-wwm-ext',
    num_labels=4,
    args=model_args_5
) 
# Train the model
model_5.train_model(train_df_5)

In [21]:
model_0 = ClassificationModel('bert', 'outputs_0', num_labels=4)
model_1 = ClassificationModel('bert', 'outputs_1', num_labels=4)
model_2 = ClassificationModel('bert', 'outputs_2', num_labels=4)
model_3 = ClassificationModel('bert', 'outputs_3', num_labels=4)
model_4 = ClassificationModel('bert', 'outputs_4', num_labels=4)
model_5 = ClassificationModel('bert', 'outputs_5', num_labels=4)

In [22]:
# multi-classification
_, raw_output_0 = model_0.predict([text for text in dev_data['text'].values])
_, raw_output_1 = model_1.predict([text for text in dev_data['text'].values])
_, raw_output_2 = model_2.predict([text for text in dev_data['text'].values])
_, raw_output_3 = model_3.predict([text for text in dev_data['text'].values])
_, raw_output_4 = model_4.predict([text for text in dev_data['text'].values])
_, raw_output_5 = model_5.predict([text for text in dev_data['text'].values])


INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/6782 [00:00<?, ?it/s]

  0%|          | 0/848 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/6782 [00:00<?, ?it/s]

...

  0%|          | 0/6782 [00:00<?, ?it/s]

  0%|          | 0/848 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/6782 [00:00<?, ?it/s]

  0%|          | 0/848 [00:00<?, ?it/s]

In [23]:
import numpy as np
import math

# custom function
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)

raw_output_0 = sigmoid_v(raw_output_0).tolist()
raw_output_1 = sigmoid_v(raw_output_1).tolist()
raw_output_2 = sigmoid_v(raw_output_2).tolist()
raw_output_3 = sigmoid_v(raw_output_3).tolist()
raw_output_4 = sigmoid_v(raw_output_4).tolist()
raw_output_5 = sigmoid_v(raw_output_5).tolist()


In [26]:

# eval on joint model
predictions, raw_outputs = model.predict([text for text in dev_data['text'].values])
raw_outputs[raw_outputs<=0.01] = 0

from sklearn.metrics import mean_squared_error
result = raw_outputs.tolist()
for i in range(len(result)):
    for j in range(6):
        if result[i][j] > 0.90:
            if j == 0:
                max_prob = max(raw_output_0[i])
                max_indx = raw_output_0[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 1:
                max_prob = max(raw_output_1[i])
                max_indx = raw_output_1[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 2:
                max_prob = max(raw_output_2[i])
                max_indx = raw_output_2[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 3:
                max_prob = max(raw_output_3[i])
                max_indx = raw_output_3[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 4:
                max_prob = max(raw_output_4[i])
                max_indx = raw_output_4[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 5:
                max_prob = max(raw_output_5[i])
                max_indx = raw_output_5[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
test_y = dev_data['labels'].values.tolist()
rmse = np.sqrt(mean_squared_error(result, test_y))
score = 1.0 / (1+rmse)
print(f"得分：{score}")

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/6782 [00:00<?, ?it/s]

  0%|          | 0/848 [00:00<?, ?it/s]

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [25]:
# multi-classification
_, raw_output_0 = model_0.predict([text for text in test['text'].values])
_, raw_output_1 = model_1.predict([text for text in test['text'].values])
_, raw_output_2 = model_2.predict([text for text in test['text'].values])
_, raw_output_3 = model_3.predict([text for text in test['text'].values])
_, raw_output_4 = model_4.predict([text for text in test['text'].values])
_, raw_output_5 = model_5.predict([text for text in test['text'].values])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/21376 [00:00<?, ?it/s]

  0%|          | 0/2672 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/21376 [00:00<?, ?it/s]

...

  0%|          | 0/21376 [00:00<?, ?it/s]

  0%|          | 0/2672 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/21376 [00:00<?, ?it/s]

  0%|          | 0/2672 [00:00<?, ?it/s]

In [27]:
raw_output_0 = sigmoid_v(raw_output_0).tolist()
raw_output_1 = sigmoid_v(raw_output_1).tolist()
raw_output_2 = sigmoid_v(raw_output_2).tolist()
raw_output_3 = sigmoid_v(raw_output_3).tolist()
raw_output_4 = sigmoid_v(raw_output_4).tolist()
raw_output_5 = sigmoid_v(raw_output_5).tolist()

In [30]:

# test
predictions, raw_outputs = model.predict([text for text in test['text'].values])
raw_outputs[raw_outputs<=0.01] = 0
# sub probs
prob_submit = submit.copy()
to_submit = []
result = raw_outputs.tolist()
for i in range(len(result)):
    for j in range(6):
        if result[i][j] > 0.95:
            if j == 0:
                max_prob = max(raw_output_0[i])
                max_indx = raw_output_0[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 1:
                max_prob = max(raw_output_1[i])
                max_indx = raw_output_1[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 2:
                max_prob = max(raw_output_2[i])
                max_indx = raw_output_2[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 3:
                max_prob = max(raw_output_3[i])
                max_indx = raw_output_3[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 4:
                max_prob = max(raw_output_4[i])
                max_indx = raw_output_4[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
            elif j == 5:
                max_prob = max(raw_output_5[i])
                max_indx = raw_output_5[i].index(max_prob)
                if max_indx == 0:
                    continue
                result[i][j] = max_prob * max_indx
# prob_submit['emotion'] = raw_outputs
for i in range(len(result)):
    lst = result[i]
    to_submit.append(','.join([str(i) for i in lst]))
    # break
prob_submit['emotion'] = to_submit #prob_submit['emotion'].apply(lambda x: ','.join([str(i) for i in x]))
prob_submit.head()
prob_submit.to_csv('0922_s6905_bert_fusion.tsv', sep='\t', index=False)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/21376 [00:00<?, ?it/s]

  0%|          | 0/2672 [00:00<?, ?it/s]

In [ ]:
print('a')